# 🚀 Enhanced Colab Processor with Plot Support
Captures matplotlib plots and other visualizations

In [ ]:
# Setup
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client matplotlib -q

from google.colab import userdata
import json
import time
import io
import os
import base64
import tempfile
import traceback
from contextlib import redirect_stdout
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Matplotlib setup
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Get service account from Colab secret (lowercase!)
sa_info = json.loads(userdata.get('sa_info'))
FOLDER_ID = sa_info.get('folder_id', '1S0gP-mWLQmnd060Atf8F2LpqEAZOdCjH')

# Create Drive service
creds = service_account.Credentials.from_service_account_info(
    sa_info,
    scopes=['https://www.googleapis.com/auth/drive']
)
drive_service = build('drive', 'v3', credentials=creds)

print('✅ Enhanced processor initialized')
print(f'📁 Monitoring folder: {FOLDER_ID}')

In [ ]:
# Main processor loop with plot capture
print('🚀 Starting enhanced processor...')
processed_commands = set()

def execute_with_plots(code):
    """Execute code and capture output + plots"""
    stdout_buffer = io.StringIO()
    initial_figs = set(plt.get_fignums())
    
    # Execute
    error = None
    try:
        with redirect_stdout(stdout_buffer):
            exec(code, {'__name__': '__main__'})
    except Exception as e:
        error = traceback.format_exc()
    
    # Capture plots
    plots = []
    new_figs = set(plt.get_fignums()) - initial_figs
    for fig_num in new_figs:
        fig = plt.figure(fig_num)
        buf = io.BytesIO()
        fig.savefig(buf, format='png', bbox_inches='tight', dpi=150)
        buf.seek(0)
        plots.append({
            'type': 'image/png',
            'data': base64.b64encode(buf.read()).decode('utf-8')
        })
        plt.close(fig)
    
    return {
        'status': 'error' if error else 'success',
        'output': stdout_buffer.getvalue(),
        'error': error,
        'visualizations': plots,
        'output_type': 'rich' if plots else 'text'
    }

while True:
    try:
        # Find command files
        query = f"'{FOLDER_ID}' in parents and name contains 'command_' and trashed=false"
        results = drive_service.files().list(q=query, fields='files(id, name)').execute()
        
        for file in results.get('files', []):
            # Read command
            content = drive_service.files().get_media(fileId=file['id']).execute()
            command = json.loads(content.decode('utf-8'))
            
            if command['id'] in processed_commands:
                continue
                
            print(f"📥 Processing: {command['id']}")
            
            # Delete command file
            try:
                drive_service.files().delete(fileId=file['id']).execute()
            except:
                pass
            
            # Execute with plot capture
            result = execute_with_plots(command.get('code', ''))
            result['command_id'] = command['id']
            
            # Write result
            with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
                json.dump(result, f, indent=2)
                temp_path = f.name
            
            try:
                media = MediaFileUpload(temp_path, mimetype='application/json')
                drive_service.files().create(
                    body={
                        'name': f"result_{command['id']}.json",
                        'parents': [FOLDER_ID]
                    },
                    media_body=media
                ).execute()
            finally:
                os.unlink(temp_path)
            
            processed_commands.add(command['id'])
            
            if result.get('visualizations'):
                print(f"✅ Done! Captured {len(result['visualizations'])} plots")
            else:
                print(f"✅ Done!")
            
        time.sleep(1)
        
    except KeyboardInterrupt:
        print("\n👋 Stopped")
        break
    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()
        time.sleep(5)